In [2]:
import openpyxl as xls
from scipy.stats import ttest_ind
from scipy.stats.contingency import chi2_contingency
import numpy as np
import re

In [3]:
import pandas as pd

In [4]:
bdc_data = pd.read_excel('230911 1600 분당차 데이터셋 김주연.xlsx', sheet_name='8672', header=11)

In [5]:
bdc_data

,year,Hospital,TYPE1,환자번호_2,환자번호_2.1,가명처리,분만일자_4,분만주_5,분만일_6,EDC_7,...,두번째 출생체중,두번째 합병증,두번째 COMMENT,세번째 분만년도,세번째 분만임신주수,세번째 분만형태,세번째 성별,세번째 출생체중,세번째 합병증,세번째 COMMENT
0,2015,분당차,정상,11427012,11427012,강*진,2015-06-03,40,6,2015-05-28,...,NaN,NaN,현재 임신중,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,분당차,정상,11418357,11418357,강*영,2015-06-09,39,0,2015-06-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,분당차,정상,11334675,11334675,강*사,2015-04-23,37,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,분당차,정상,11545481,11545481,강*화,2015-09-03,39,0,2015-09-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,분당차,정상,11584851,11584851,강*아,2015-05-28,38,1,2015-06-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8667,2022,분당차,분만,11479420,11479420,황*혜,2022-09-06,38,1,2022-09-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8668,2022,분당차,분만,12161785,12161785,황*아,2022-03-22,39,0,2022-03-29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8669,2022,분당차,분만,12137290,12137290,황*신,2022-01-23,40,1,2022-01-22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8670,2022,분당차,분만,12126800,12126800,후*모토마이코,2022-02-03,39,1,2022-02-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
cases = bdc_data.loc[bdc_data['임신중독증 유무']=='유']
conts = bdc_data.loc[bdc_data['임신중독증 유무']=='무']
len(cases)+len(conts)

8672

In [7]:
def __find_name_of_col_bundanglogicnum__(x, tar_str):
    res = re.findall('[^0-9]_'+tar_str+'(?:$|[^0-9])',x)
    if len(res)<1:
        return False
    elif (res[0][2:]==tar_str)|(res[0][2:-1]==tar_str):
        return True
    else:
        return False
def find_name_of_col_bundanglogicnum(logicnum):
    i_col = np.where([__find_name_of_col_bundanglogicnum__(x,logicnum) for x in bdc_data.columns])
    name_col = bdc_data.columns[i_col]
    return name_col
# print(find_name_of_col_bundanglogicnum('16_1'))
k=1.96
def analy_for_conti_values(cases_0, conts_0, ):
    cases_ = cases_0.dropna()
    conts_ = conts_0.dropna()
    print('Missings:', len(cases_0)+len(conts_0)-len(cases_)-len(conts_))
    case_m, case_sd, cont_m, cont_sd = np.mean(cases_),np.std(cases_),np.mean(conts_),np.std(conts_)
    print('{:.2f} ({:.2f},{:.2f})'.format(case_m, case_m-k*case_sd, case_m+k*case_sd), end=' ')
    print('{:.2f} ({:.2f},{:.2f})'.format(cont_m, cont_m-k*cont_sd, cont_m+k*cont_sd), end=' ')
    print('{:.4f}'.format(ttest_ind(list(cases_), list(conts_))[1]))
def analy_for_cate_values(cases_0, conts_0, ):
    cases_ = cases_0.dropna()
    conts_ = conts_0.dropna()
    print('Missings:', len(cases_0)+len(conts_0)-len(cases_)-len(conts_))
    case0, case1, cont0, cont1 = sum(cases_==0), sum(cases_==1), sum(conts_==0), sum(conts_==1)
    obs = np.zeros(shape=(2,2))
    obs[0,0] = case0; obs[0,1] = cont0;  
    obs[1,0] = case1; obs[1,1] = cont1; 
    obs = obs.astype(np.int64)
    print(obs)
    print("{:.3f}".format(chi2_contingency(obs)[1]))

In [8]:
col_names = find_name_of_col_bundanglogicnum('9')
print(col_names); selected_col = col_names[0]
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print('************************************')
analy_for_conti_values(cases_, conts_)

Index(['만나이분만일기준_9'], dtype='object')
************************************
Missings: 0
35.38 (26.93,43.83) 34.46 (26.46,42.46) 0.0020


In [9]:
col_names = find_name_of_col_bundanglogicnum('16_1')
print(col_names); selected_col = col_names[1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if x=='자연임신':
        return 0
    return 1
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['이번임신방법_16_1', '이번임신방법_16_1.1'], dtype='object')
[nan '시술_체외수정(IVF)' '시술_인공수정(IUI)' '자연임신' '체외수정(IVF)' '인공수정(IUI)'
 '배란유도제사용']
************************************
Missings: 5517
[[  43 1543]
 [  46 1523]]
0.790


In [10]:
col_names = find_name_of_col_bundanglogicnum('16_1') #엑셀 표에는 16_2를 참조하라고 했으나 16_1을 기준으로 하는 게 맞다고 판단하였음.
print(col_names)
selected_col = col_names[1]
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if 'IVF' in x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['이번임신방법_16_1', '이번임신방법_16_1.1'], dtype='object')
************************************
Missings: 5517
[[  52 1795]
 [  37 1271]]
1.000


In [11]:
col_names = find_name_of_col_bundanglogicnum('35') 
print(col_names)
selected_col = col_names[1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if '유'==x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['흡연력_35', '흡연력_35.1'], dtype='object')
['무' nan '유']
************************************
Missings: 17
[[ 189 8414]
 [   1   51]]
1.000


In [12]:
col_names = find_name_of_col_bundanglogicnum('36')
print(col_names); selected_col = col_names[1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if '유'==x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['음주력_36', '음주력_36.1'], dtype='object')
['무' nan '유']
************************************
Missings: 17
[[ 189 8438]
 [   1   27]]
1.000


In [13]:
col_names = find_name_of_col_bundanglogicnum('52')
print(col_names)
selected_col = col_names[0]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if '유'==x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['고혈압_52'], dtype='object')
[nan '무' '유']
************************************
Missings: 7027
[[  40 1578]
 [   7   20]]
0.000


In [80]:
col_names = find_name_of_col_bundanglogicnum('437')
print(col_names); selected_col = col_names[-1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if '유' in x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['과거력당뇨_437', '과거력당뇨_437(류)', '과거력당뇨_437(김)', '과거력당뇨_437_(차트리뷰반영)'], dtype='object')
['무' nan '유' '무(출산력변수)' '유(출산력변수)' '무 ']
************************************
Missings: 30
[[ 175 8367]
 [  13   87]]
0.000


In [15]:
selected_col = 'para 정보 가져오기'
print(selected_col)
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif (x=='0(G1)'):
        return 0
    elif type(x)==str:
        return 1
    elif x > 0:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
cases_para_indexes = cases__[cases__==1].index
conts_para_indexes = conts__[conts__==1].index
print('************************************')
analy_for_cate_values(cases__, conts__)

para 정보 가져오기
[1 0 2 3 4 '0->1' 5 '0->2' '0(G1)' 8]
************************************
Missings: 0
[[ 128 5241]
 [  64 3239]]
0.195


In [16]:
selected_col = '8.21\n자연조산 재분류'
print(selected_col)
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if x=='자연조산아님':
        return -1
    if x=='자연조산':
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
cases__ = cases__.loc[cases_para_indexes]
conts__ = conts__.loc[conts_para_indexes]
print('************************************')
print(sum(cases__==-1)+sum(conts__==-1))
analy_for_cate_values(cases__, conts__)

8.21
자연조산 재분류
['만삭분만' '만삭분만_자연진통' '자연조산' '자연조산아님']
************************************
92
Missings: 0
[[  32 2905]
 [  24  250]]
0.000


In [17]:
col_names = find_name_of_col_bundanglogicnum('39')
print(col_names)
selected_col = col_names[-1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if not pd.isna(x):
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
cases__ = cases__.loc[cases_para_indexes]
conts__ = conts__.loc[conts_para_indexes]
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['임신성고혈압질환과거력_39'], dtype='object')
[nan '유' '유(출산력변수)']
************************************
Missings: 0
[[  35 3135]
 [  29  104]]
0.000


In [18]:
col_names = find_name_of_col_bundanglogicnum('40')
print(col_names)
selected_col = col_names[-1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif '유' in x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
cases__ = cases__.loc[cases_para_indexes]
conts__ = conts__.loc[conts_para_indexes]
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['임신성당뇨_40', '임신성당뇨_40.1'], dtype='object')
[nan '유' '유(출산력변수)' '무' '무(출산력변수)']
************************************
Missings: 3010
[[  5  33]
 [ 11 244]]
0.063


In [19]:
col_names = find_name_of_col_bundanglogicnum('424')
print(col_names)
selected_col = col_names[-1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
cases__ = pd.Series(0, index=cases_.index);
conts__ = pd.Series(0, index=conts_.index)
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif '유' in x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['가족력당뇨_424', '가족력당뇨_424.1', '가족력당뇨_424.2', '가족력당뇨_424.3'], dtype='object')
['무' '유' nan]
************************************
Missings: 3
[[ 136 6928]
 [  56 1549]]
0.000


In [20]:
col_names = find_name_of_col_bundanglogicnum('426')
print(col_names)
selected_col = col_names[-1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif '유' in x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['가족력고혈압_426', '가족력고혈압_426.1', '가족력고혈압_426.2'], dtype='object')
['무' '유' nan]
************************************
Missings: 3
[[ 103 6084]
 [  89 2393]]
0.000


In [21]:
col_names = find_name_of_col_bundanglogicnum('90')
print(col_names); selected_col = col_names[0]
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if ((x<15) or (x>50)):
        return np.nan
    return np.float32(x)
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

Index(['BMI_90', 'BMI_90.1'], dtype='object')
************************************
Missings: 6952
25.76 (15.21,36.32) 22.41 (15.25,29.56) 0.0000


In [22]:
col_names = find_name_of_col_bundanglogicnum('90')
print(col_names); selected_col = col_names[0]
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if ((x<15) or (x>50)):
        return np.nan
    if x>=30:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['BMI_90', 'BMI_90.1'], dtype='object')
************************************
Missings: 6952
[[  40 1590]
 [  12   78]]
0.000


In [23]:
col_names = find_name_of_col_bundanglogicnum('88')
print(col_names)
selected_col = col_names[1]
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print('************************************')
analy_for_conti_values(cases_, conts_)

Index(['혈압수축기_88', '혈압수축기_88.1'], dtype='object')
************************************
Missings: 6295
136.39 (98.58,174.19) 117.55 (90.53,144.57) 0.0000


In [24]:
col_names = find_name_of_col_bundanglogicnum('89')
print(col_names)
selected_col = col_names[1]
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print('************************************')
analy_for_conti_values(cases_, conts_)

Index(['혈압이완기_89', '혈압이완기_89.1'], dtype='object')
************************************
Missings: 6296
84.26 (54.25,114.26) 69.28 (47.74,90.82) 0.0000


In [86]:
col_names = find_name_of_col_bundanglogicnum('88')
print(col_names)
selected_col = col_names[1]
cases_, conts_ = cases[selected_col], conts[selected_col]
col_names = find_name_of_col_bundanglogicnum('89')
print(col_names)
selected_col = col_names[1]
cases2_, conts2_ = cases[selected_col], conts[selected_col]
print('************************************')
analy_for_conti_values((cases_+cases2_*2)/3, (conts_+conts2_*2)/3)

Index(['혈압수축기_88', '혈압수축기_88.1'], dtype='object')
Index(['혈압이완기_89', '혈압이완기_89.1'], dtype='object')
************************************
Missings: 6296
101.63 (70.87,132.40) 85.37 (63.76,106.98) 0.0000


In [26]:
selected_col = 'BMI11_94'
print(selected_col)
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    x = np.float32(x)
    if ((x<15) or (x>50)):
        return np.nan
    return x
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

BMI11_94
************************************
Missings: 5616
26.62 (14.57,38.68) 22.73 (15.54,29.93) 0.0000


In [27]:
selected_col = 'BMI11_94'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    x = np.float32(x)
    if ((x<15) or (x>50)):
        return np.nan
    if x>=30:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

BMI11_94
************************************
Missings: 5616
[[  66 2818]
 [  18  154]]
0.000


In [28]:
selected_col = '혈압수축기11_91'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

혈압수축기11_91
************************************
Missings: 2932
126.95 (101.85,152.04) 113.46 (89.77,137.15) 0.0000


In [29]:
selected_col = '혈압이완기11_92'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

혈압이완기11_92
************************************
Missings: 2932
79.96 (55.96,103.95) 69.63 (46.92,92.34) 0.0000


In [82]:
selected_col = '혈압이완기11_92'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
selected_col = '혈압수축기11_91'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
cases2__ = cases_.apply(lambda x: tempfunc_(x))
conts2__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_conti_values((cases__*2+cases2__)/3, (conts__*2+conts2__)/3)

혈압이완기11_92
혈압수축기11_91
************************************
Missings: 2933
95.62 (72.89,118.35) 84.24 (63.85,104.63) 0.0000


In [31]:
selected_col = 'BMI16_98'
print(selected_col)
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    x = np.float32(x)
    if ((x<15) or (x>50)):
        return np.nan
    return x
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

BMI16_98
************************************
Missings: 5901
26.74 (17.46,36.01) 23.39 (16.22,30.56) 0.0000


In [32]:
selected_col = 'BMI16_98'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    x = np.float32(x)
    if ((x<15) or (x>50)):
        return np.nan
    if x>=30:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

BMI16_98
************************************
Missings: 5901
[[  60 2528]
 [  19  164]]
0.000


In [33]:
selected_col = '혈압수축기16_95'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

혈압수축기16_95
************************************
Missings: 2168
125.34 (100.25,150.42) 112.42 (89.36,135.48) 0.0000


In [34]:
selected_col = '혈압이완기16_96'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

혈압이완기16_96
************************************
Missings: 2170
78.72 (55.38,102.06) 67.46 (35.27,99.66) 0.0000


In [83]:
selected_col = '혈압이완기16_96'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
selected_col = '혈압수축기16_95'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
cases2__ = cases_.apply(lambda x: tempfunc_(x))
conts2__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_conti_values((cases__*2+cases2__)/3, (conts__*2+conts2__)/3)

혈압이완기16_96
혈압수축기16_95
************************************
Missings: 2170
94.26 (71.76,116.76) 82.45 (57.05,107.85) 0.0000


In [36]:
selected_col = 'BMI20_101'
print(selected_col)
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    x = np.float32(x)
    if ((x<15) or (x>50)):
        return np.nan
    return x
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

BMI20_101
************************************
Missings: 5589
27.44 (17.61,37.28) 24.26 (17.11,31.41) 0.0000


In [37]:
selected_col = 'BMI20_101'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    x = np.float32(x)
    if ((x<15) or (x>50)):
        return np.nan
    if x>=30:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

BMI20_101
************************************
Missings: 5589
[[  61 2766]
 [  25  231]]
0.000


In [38]:
selected_col = '혈압수축기20_99_1'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

혈압수축기20_99_1
************************************
Missings: 1831
126.32 (104.62,148.03) 112.45 (89.55,135.35) 0.0000


In [39]:
selected_col = '혈압이완기20_99_2'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

혈압이완기20_99_2
************************************
Missings: 1831
78.20 (54.78,101.62) 66.95 (39.61,94.28) 0.0000


In [84]:
selected_col = '혈압이완기20_99_2'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
selected_col = '혈압수축기20_99_1'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
cases2__ = cases_.apply(lambda x: tempfunc_(x))
conts2__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_conti_values((cases__*2+cases2__)/3, (conts__*2+conts2__)/3)

혈압이완기20_99_2
혈압수축기20_99_1
************************************
Missings: 1832
94.24 (72.88,115.60) 82.11 (59.55,104.67) 0.0000


In [41]:
selected_col = '함+산\nPAPP-A(MoM)'
print(selected_col)
a = np.where(bdc_data.columns==selected_col)[0][0]
for i in range(1,20):
    if '차주' in bdc_data.columns[a-i]:
        print(bdc_data.columns[a-i]);week_i = a-i
        break
print(bdc_data[selected_col].unique())
print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif x == ' ':
        return False
    elif (float(x)<=22)&(float(x)>0):
        return True
    return False
cases_, conts_ = cases[cases.iloc[:,week_i].apply(lambda x: func_for_week(x))], conts[conts.iloc[:,week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

함+산
PAPP-A(MoM)
INTEGRATED1차주_122
[0.504   nan 1.263 ... 2.557 2.184 2.628]
['11' nan '12' '-65' '13' '10' '14' '-116' '-45' '-82' '-238' '-273' '15'
 '-93' '-134' '-172' '-178' '-145' 11 12 13 10 -117 ' ' -46 -77 -38 '-52']
************************************
5559
Missings: 48
1.04 (-0.24,2.32) 1.19 (-0.02,2.40) 0.0117


In [42]:
selected_col = '함+산\nAFP MoM'
print(selected_col)
a = np.where(bdc_data.columns==selected_col)[0][0]
for i in range(1,20):
    if '차주' in bdc_data.columns[a-i]:
        print(bdc_data.columns[a-i]);week_i = a-i
        break
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[cases.iloc[:,week_i].apply(lambda x: func_for_week(x))], conts[conts.iloc[:,week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

함+산
AFP MoM
INTEGRAGTED2차주_146
[0.806   nan 0.913 ... 1.506 1.193 2.467]
************************************
5714
Missings: 8
1.19 (0.29,2.08) 1.07 (0.27,1.87) 0.0023


In [43]:
selected_col = '함+산\nTotalhCGB MoM'
print(selected_col)
a = np.where(bdc_data.columns==selected_col)[0][0]
for i in range(1,20):
    if '차주' in bdc_data.columns[a-i]:
        print(bdc_data.columns[a-i]);week_i = a-i
        break
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[cases.iloc[:,week_i].apply(lambda x: func_for_week(x))], conts[conts.iloc[:,week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

함+산
TotalhCGB MoM
INTEGRAGTED2차주_146
[0.878   nan 0.787 ... 0.294 3.119 3.134]
************************************
5714
Missings: 8
1.62 (-0.27,3.51) 1.17 (-0.04,2.39) 0.0000


In [44]:
selected_col = '함+산\nUE3 MoM'
print(selected_col)
a = np.where(bdc_data.columns==selected_col)[0][0]
for i in range(1,20):
    if '차주' in bdc_data.columns[a-i]:
        print(bdc_data.columns[a-i]);week_i = a-i
        break
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[cases.iloc[:,week_i].apply(lambda x: func_for_week(x))], conts[conts.iloc[:,week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

함+산
UE3 MoM
INTEGRAGTED2차주_146
[0.666   nan 0.839 ... 1.959 1.456 1.378]
************************************
5714
Missings: 8
0.91 (0.38,1.45) 1.02 (-1.38,3.43) 0.3277


In [45]:
selected_col = '함+산\nINHIBIN MoM'
print(selected_col)
a = np.where(bdc_data.columns==selected_col)[0][0]
for i in range(1,20):
    if '차주' in bdc_data.columns[a-i]:
        print(bdc_data.columns[a-i]);week_i = a-i
        break
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[cases.iloc[:,week_i].apply(lambda x: func_for_week(x))], conts[conts.iloc[:,week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

함+산
INHIBIN MoM
INTEGRAGTED2차주_146
[0.788   nan 1.042 ... 1.637 0.521 2.018]
************************************
5714
Missings: 8
1.58 (-1.17,4.33) 1.06 (0.08,2.04) 0.0000


In [46]:
col_names = find_name_of_col_bundanglogicnum('118_1')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns==selected_col)[0][0]
for i in range(1,20):
    if '주_계산' in bdc_data.columns[a+i]:
        print(bdc_data.columns[a+i]);week_i = a+i
        break
print(bdc_data[selected_col].unique())
print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x)<=22)&(float(x)>0):
        return True
    return False
cases_, conts_ = cases[cases.iloc[:,week_i].apply(lambda x: func_for_week(x))], conts[conts.iloc[:,week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['WBC_118_1'], dtype='object')
주_계산.8
[10.77 10.53  6.67 ... 12.74 13.3  18.76]
[11. 35. 10.  7. 37.  9. 34. 16. 12. 36. 13. 23. 38.  6. 26. 29. 33. 19.
 22. 32. 14. 25. 39. 27.  8. 17. 15. 18. nan 20. 28. 31.  3. 24. 30. 21.
 40.  5.  4.  0. -1. -4.  2. -2.  1. -5.]
************************************
3953
Missings: 0
9.12 (4.60,13.64) 8.42 (4.34,12.49) 0.0006


In [47]:
col_names = find_name_of_col_bundanglogicnum('118_3')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns==selected_col)[0][0]
for i in range(1,20):
    if '주_계산' in bdc_data.columns[a+i]:
        print(bdc_data.columns[a+i]);week_i = a+i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x)<=22)&(float(x)>0):
        return True
    return False
cases_, conts_ = cases[cases.iloc[:,week_i].apply(lambda x: func_for_week(x))], conts[conts.iloc[:,week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['HGB_118_3'], dtype='object')
주_계산.8
************************************
3953
Missings: 0
12.62 (9.83,15.40) 12.48 (10.46,14.51) 0.1901


In [48]:
col_names = find_name_of_col_bundanglogicnum('118_4')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['HCT_118_4'], dtype='object')
주_계산.8
************************************
3953
Missings: 0
37.22 (28.89,45.54) 36.82 (30.97,42.68) 0.1857


In [49]:
col_names = find_name_of_col_bundanglogicnum('118_9')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['PLT_118_9'], dtype='object')
주_계산.8
************************************
3953
Missings: 0
268.35 (125.74,410.96) 249.06 (138.14,359.97) 0.0006


In [50]:
col_names = find_name_of_col_bundanglogicnum('118_30')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
print(bdc_data[selected_col].unique())
print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if x is np.nan:
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['TSH_118_30', 'TSH_118_30.1', 'TSH_118_30.2'], dtype='object')
주_계산.20
['0.842' nan '0.582' ... 1.243 2.072 0.302]
[11. nan  7. -1. 12. 16.  6. 17. 34. 19. 15.  9.  8. 13. 10. 25.  3. 22.
 18. 26. 14. 20. 36. 38. 30. 27. 21. 23. 29.  0. 28. 35. 24. 37.  1. 31.
  4. 39.  5. 32.  2. 33. 40. -8.]
************************************
4034
Missings: 68
1.22 (-0.90,3.33) 1.32 (-1.69,4.33) 0.5032


In [51]:
selected_col = 'FT4_118_31.2'
print(selected_col)
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

FT4_118_31.2
주_계산.21
************************************
2942
Missings: 768
1.22 (0.89,1.55) 1.23 (0.75,1.72) 0.6658


In [52]:
col_names = find_name_of_col_bundanglogicnum('118_33')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['BUN_118_33', 'BUN_118_33.1', 'BUN_118_33.2'], dtype='object')
주_계산.22
************************************
3032
Missings: 2
8.43 (3.12,13.74) 7.98 (3.56,12.39) 0.0646


In [53]:
col_names = find_name_of_col_bundanglogicnum('118_34')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['CREATININE_118_34', 'CREATININE_118_34.1', 'CREATININE_118_34.2'], dtype='object')
주_계산.23
************************************
2501
Missings: 0
0.53 (0.30,0.77) 0.52 (0.30,0.73) 0.1634


In [54]:
col_names = find_name_of_col_bundanglogicnum('118_35')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['GLUCOSE_118_35', 'GLUCOSE_118_35.1', 'GLUCOSE_118_35.2'], dtype='object')
주_계산.24
************************************
621
Missings: 0
104.67 (41.61,167.74) 95.76 (48.11,143.42) 0.0292


In [55]:
col_names = find_name_of_col_bundanglogicnum('118_36')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['HGBA1C_118_36', 'HGBA1C_118_36.1', 'HGBA1C_118_36.2',
       'HGBA1C_118_36.3'],
      dtype='object')
주_계산.26
************************************
3544
Missings: 2586
5.75 (4.12,7.38) 5.32 (4.18,6.46) 0.0000


In [56]:
col_names = find_name_of_col_bundanglogicnum('118_37')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
#print(bdc_data[selected_col].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['25OHVitaminD_118_37', '25OHVitaminD_118_37.1', '25OHVitaminD_118_37.2',
       '25OHVitaminD_118_37.3'],
      dtype='object')
주_계산.27
************************************
673
Missings: 199
18.59 (-2.82,40.00) 18.63 (-1.68,38.95) 0.9859


In [57]:
col_names = find_name_of_col_bundanglogicnum('118_38')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['HDLCHOLESTEROL_118_38', 'HDLCHOLESTEROL_118_38.1',
       'HDLCHOLESTEROL_118_38.2', 'HDLCHOLESTEROL_118_38.3'],
      dtype='object')
주_계산.29
************************************
1290
Missings: 497
65.79 (33.75,97.83) 71.03 (37.12,104.94) 0.0798


In [58]:
col_names = find_name_of_col_bundanglogicnum('118_39')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['TRIGLYCERIDE_118_39', 'TRIGLYCERIDE_118_39.1',
       'TRIGLYCERIDE_118_39.2'],
      dtype='object')
주_계산.30
************************************
2675
Missings: 197
229.10 (-117.01,575.21) 223.85 (-18.48,466.18) 0.7302


In [59]:
col_names = find_name_of_col_bundanglogicnum('118_40')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['T.Protein_118_40', 'T.Protein_118_40.1', 'T.Protein_118_40.2'], dtype='object')
주_계산.31
************************************
2755
Missings: 17
6.92 (5.94,7.89) 6.81 (5.95,7.68) 0.0494


In [60]:
col_names = find_name_of_col_bundanglogicnum('118_41')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['ALBUMIN_118_41', 'ALBUMIN_118_41.1', 'ALBUMIN_118_41.2'], dtype='object')
주_계산.32
************************************
2843
Missings: 2
4.06 (3.41,4.71) 4.10 (3.49,4.72) 0.2297


In [61]:
col_names = find_name_of_col_bundanglogicnum('118_42')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['T.Bilirubin_118_42', 'T.Bilirubin_118_42.1', 'T.Bilirubin_118_42.2'], dtype='object')
주_계산.33
************************************
4448
Missings: 200
0.41 (-1.20,2.02) 0.32 (-0.03,0.68) 0.0000


In [62]:
col_names = find_name_of_col_bundanglogicnum('118_43')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['ASGOT_118_43', 'ASGOT_118_43.1', 'ASGOT_118_43.2'], dtype='object')
주_계산.34
************************************
4450
Missings: 0
18.21 (2.04,34.38) 17.12 (0.08,34.15) 0.1837


In [63]:
col_names = find_name_of_col_bundanglogicnum('118_44')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['ALTGPT_118_44', 'ALTGPT_118_44.1', 'ALTGPT_118_44.2'], dtype='object')
주_계산.35
************************************
2657
Missings: 0
21.40 (-14.12,56.93) 16.99 (-15.95,49.92) 0.0283


In [64]:
col_names = find_name_of_col_bundanglogicnum('118_45')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['ALP_118_45', 'ALP_118_45.1', 'ALP_118_45.2'], dtype='object')
주_계산.36
************************************
2843
Missings: 18
61.02 (11.37,110.68) 65.58 (-4.80,135.96) 0.2570


In [65]:
col_names = find_name_of_col_bundanglogicnum('118_46')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['TOTALCHOLESTEROL_118_46', 'TOTALCHOLESTEROL_118_46.1',
       'TOTALCHOLESTEROL_118_46.2'],
      dtype='object')
주_계산.37
************************************
6614
Missings: 547
218.98 (125.85,312.12) 222.42 (129.17,315.67) 0.3682


In [66]:
selected_col = '08.21\n임당군/정상군 분류'
print(selected_col)
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print(cases_.unique(), conts_.unique())
print('************************************')
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if x=='GDM':
        return 1
    if x=='정상군':
        return 0
    return -1
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
print('제외군', sum(cases__==-1)+sum(cases__==-1))
analy_for_cate_values(cases__, conts__)

08.21
임당군/정상군 분류
['제외군(overtDM)' '제외군(검사안함)' '정상군' 'GDM' '제외군(정상군 -> overtDM)'] ['정상군' 'GDM' '제외군(검사안함)' '제외군(overtDM)' '제외군(GDM-> overDM)'
 '제외군(정상군 -> overtDM)' '제외군(검사안함) -> overtDM']
************************************
************************************
제외군 66
Missings: 0
[[ 131 7135]
 [  28  812]]
0.004


In [67]:
col_names = find_name_of_col_bundanglogicnum('5')
print(col_names)
selected_col = col_names[-1]
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases_.apply(lambda x: tempfunc(x)), conts_.apply(lambda x: tempfunc(x))
print('************************************')
analy_for_conti_values(cases_, conts_)

Index(['분만주_5'], dtype='object')
************************************
Missings: 0
35.54 (29.15,41.92) 38.07 (34.43,41.72) 0.0000


In [68]:
col_names = find_name_of_col_bundanglogicnum('5')
print(col_names)
selected_col = col_names[-1]
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print(cases_.unique(), conts_.unique())
print('************************************')
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif x<=37:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['분만주_5'], dtype='object')
[36 31 37 40 35 38 39 34 29 33 32 26 28 27 30 24 25 22] [40 39 37 38 24 36 33 22 34 27 35 25 41 20 32 23 30 31 26 29 28 21 42]
************************************
************************************
Missings: 0
[[  54 6507]
 [ 138 1973]]
0.000


In [69]:
col_names = find_name_of_col_bundanglogicnum('334')
print(col_names)
selected_col = col_names[-1]
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases_.apply(lambda x: tempfunc(x)), conts_.apply(lambda x: tempfunc(x))
print('************************************')
analy_for_conti_values(cases_, conts_)

Index(['WTGM_334'], dtype='object')
************************************
Missings: 0
2474.19 (937.59,4010.80) 3136.75 (2211.40,4062.10) 0.0000


In [70]:
col_names = find_name_of_col_bundanglogicnum('334')
print(col_names)
selected_col = col_names[-1]
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print('************************************')
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif x>=4000:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['WTGM_334'], dtype='object')
************************************
************************************
Missings: 0
[[ 189 8317]
 [   3  163]]
0.926


In [71]:
col_names = find_name_of_col_bundanglogicnum('334')
print(col_names)
selected_col = col_names[-1]
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
#print(cases_.unique(), conts_.unique())
print('************************************')
def tempfunc_(x):
    if pd.isna(x):
        return False
    elif x<=2500:
        return True
    return False
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['WTGM_334'], dtype='object')
************************************
************************************
Missings: 0
[[ 106 7940]
 [  86  540]]
0.000


In [72]:
selected_col = 'fenton curve\n(weight %)_674'
print(selected_col); print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
#print(cases_.unique(), conts_.unique())
print('************************************')
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if x=='성별 X':
        return np.nan
    elif float(x)>=0.9:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

fenton curve
(weight %)_674
[0.16111869103726667 0.6739711942982225 0.5949395466936785 ...
 0.38021437092800264 0.31399242962411256 0.6492741333970126]
************************************
************************************
Missings: 17
[[ 181 8190]
 [  10  274]]
0.184


In [73]:
selected_col = 'fenton curve\n(weight %)_674'
print(selected_col)
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if x=='성별 X':
        return np.nan
    elif float(x)<=0.1:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

fenton curve
(weight %)_674
************************************
Missings: 17
[[ 162 7875]
 [  29  589]]
0.000


In [74]:
selected_col = '1분 7점 미만 (1)'
print(selected_col)
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print(cases_.unique(), conts_.unique())
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif x==1:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

1분 7점 미만 (1)
[ 0.  1. nan] [ 0.  1. nan]
************************************
Missings: 1852
[[  83 6439]
 [  41  257]]
0.000


In [75]:
selected_col = '5분7점 미만(1)'
print(selected_col)
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
#print(cases_.unique(), conts_.unique())
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif x==1:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

5분7점 미만(1)
************************************
Missings: 1248
[[ 130 7179]
 [  10  105]]
0.000


In [76]:
col_names = find_name_of_col_bundanglogicnum('338')
print(col_names); selected_col = col_names[-1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if x=='Y':
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['NICU입원여부_338'], dtype='object')
[nan 'Y']
************************************
Missings: 0
[[ 107 7302]
 [  85 1178]]
0.000


In [77]:
col_names = find_name_of_col_bundanglogicnum('349')
print(col_names)
selected_col = col_names[-1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif x=='Error':
        return np.nan
    return np.float32(x)
cases_, conts_ = cases_.apply(lambda x: tempfunc(x)), conts_.apply(lambda x: tempfunc(x))
print('************************************')
analy_for_conti_values(cases_, conts_)

Index(['BST_349'], dtype='object')
[nan '82' '66' '74' '62' '72' '114' '60' '63' '58' '94' '65' '70' '29'
 '61' '81' '95' '47' '49' '57' '87' '77' '64' '89' '35' '48' '75' '52'
 '79' '68' '54' '45' '55' '69' '46' '97' '73' '44' '102' '83' '34' '42'
 '86' '67' '21' '80' '59' '56' '41' '23' '76' '98' '39' '107' '84' '92'
 '38' '71' '85' '101' '53' '78' '90' '24' '120' '88' '91' '50' '103' '51'
 '43' '100' '93' '113' '36' '104' '96' '111' '99' '112' '28' '109' '106'
 '25' 'Error' '40' '30' '31' '115' '33' '32' 73 64 60 58 67 76 78 81 57 63
 70 41 72 75 52 85 66 68 65 51 71 61 79 62 80 53 86 54 77 82 47 44 69 38
 36 29 104 43 84 50 74 45 28 59 48 88 55 35 46 56 42 92 49 34 40 89 91 83
 93 31 37 102 26 90 125 '37' '105' 30 33 25 27 127 39 103 94 24]
************************************
Missings: 6203
59.02 (28.29,89.75) 64.43 (37.62,91.25) 0.0000


In [78]:
col_names = find_name_of_col_bundanglogicnum('349')
print(col_names)
selected_col = col_names[-1]
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif x=='Error':
        return np.nan
    elif float(x)<=50:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['BST_349'], dtype='object')
************************************
Missings: 6203
[[  87 2021]
 [  34  327]]
0.000


In [79]:
col_names = find_name_of_col_bundanglogicnum('351')
print(col_names)
selected_col = col_names[-1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print('모두 nan value라 분석 불가.')

Index(['SEIZURE_351'], dtype='object')
[nan]
모두 nan value라 분석 불가.
